# Caltrans Data Extraction

The goal of this project is to extract relevant data from text files, previously converted from PDF files. Since the text files are quite structured, the decision is to use regex to do this.

## Setup

Install the following packages if you don't have them yet:

In [1]:
# pip install pandas numpy tqdm ipykernel notebook python-dotenv openpyxl

In [2]:
from utils import *

%reload_ext autoreload
%autoreload 2

# pd.set_option('display.max_rows', None)  # optional to see all rows in DataFrames

In [9]:
df_contract_types, _ = get_contract_types()
files = list(df_contract_types[df_contract_types[CONTRACT_TYPE] == 1][RELATIVE_FOLDER].values)
# files = random.sample(files, 10)
filepaths = [RAW_DATA_PATH / x for x in files]
ex = Experiment(filepaths)


In [10]:
len(ex.filepaths)

8809

In [11]:
ex.run()

  0%|          | 0/8809 [00:00<?, ?it/s]

 18%|█▊        | 1566/8809 [00:20<02:10, 55.49it/s]

{'Error_Filename': '07-0W0404..pdf_12652', 'Error': AttributeError("'NoneType' object has no attribute 'groups'")}


 34%|███▎      | 2959/8809 [00:52<02:14, 43.47it/s]

{'Error_Filename': '110427R0.pdf_3052', 'Error': AttributeError("'NoneType' object has no attribute 'groups'")}


...

 79%|███████▉  | 6989/8809 [04:00<01:26, 21.02it/s]

{'Error_Filename': '120717R0.pdf_4252', 'Error': AttributeError("'NoneType' object has no attribute 'groups'")}


 89%|████████▉ | 7847/8809 [04:57<00:57, 16.60it/s]

{'Error_Filename': '08-1G2804..pdf_12877', 'Error': AttributeError("'NoneType' object has no attribute 'groups'")}


100%|██████████| 8809/8809 [06:04<00:00, 24.14it/s]
